In [1]:
import cvxpy as cp
import numpy as np

In [2]:
# Data of the problem
cost = np.array([3]).reshape(-1,1)
all_demand = np.array([[1.5],[2],[3.5],[4],[5.5]]).reshape(-1,1)
selling_price = np.array([5]).reshape(-1,1)
salvage_price= np.array([1]).reshape(-1,1)
senario = len(all_demand)
senario_prob = (1/senario)*(np.ones_like(all_demand))

In [3]:
def solve_second_stage_subproblem(selling_price,salvage_price,supply,demand):

    sell = cp.Variable(shape=(1,1),name="sell")
    salvage = cp.Variable(shape=(1,1),name="salvage")

    objective = cp.Minimize(-1*(selling_price.T@sell)-1*(salvage_price.T@salvage))   # Objective function for second stage problem

    constraints =  [sell+salvage<=supply,sell<=demand,salvage>=0,sell>=0]  # Constriants for second stage problem

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return problem

In [4]:
def solve_master_problem(produce,g_ks,alpha_ks,cost,all_demand):
    
    # Intiatel variables for the problem
    produce  = cp.Variable(shape=(1,1),name="produce")
    v = cp.Variable(shape=(1,1),name="value")

    # Contraints for first stage problem
    constraints = []
    for cut in range(0,len(g_ks)):
        constraints.append(np.array(g_ks[cut]).T@produce+np.array(alpha_ks[cut])<=v)
    constraints.extend([produce>=0,v>=-1000000,produce<=max(all_demand)])

    objective = cp.Minimize((cost.T@produce)+v)    # Objective function for first stage problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    return problem

In [54]:
produce = np.array([5])        # Intial Guess

# Intitalize g and alpha for storing gs and alphas for each cut 
g_ks = []
alpha_ks = []

iter=0
objctive_values  = [np.nan]
epsilon = 10**(-4)
while True:

    # Solve Second stage problem for each demand and store its duals and objective values
    duals = []
    objs= []
    for demand in all_demand:

        second_stage_sol = solve_second_stage_subproblem(selling_price,salvage_price,produce,demand)

        temp_dual = second_stage_sol.constraints[0].dual_value   # Take the duals of 1st contraint
        temp_obj = second_stage_sol.value                        # Take the objective value of second stage problem

        # Store duals and objective values for each senario
        duals.append(temp_dual) 
        objs.append(temp_obj)

    
    
    # Reshaping the values 
    duals = np.array(duals).reshape(-1,1)
    objs = np.array(objs).reshape(-1,1)
   

    break


    # Computing g_ks and alpha_ks 
    g_ks_temp = -senario_prob.T@duals
    alpha_ks_temp = senario_prob.T@objs - g_ks_temp.T@produce
    
    # Store the values in data storage
    g_ks.append(g_ks_temp)
    alpha_ks.append(alpha_ks_temp)

    # Solve the first stage problem
    first_stage_sol = solve_master_problem(produce,g_ks,alpha_ks,cost,all_demand)

    obj_value = first_stage_sol.value
    new_produce = first_stage_sol.var_dict["produce"].value
    new_limit = first_stage_sol.var_dict["value"].value

    if np.abs(obj_value - objctive_values[-1])<= epsilon:
        print("Terminating condition satisfied !")
        break
    else:
        pass

    objctive_values.append(obj_value)
    produce,limit = new_produce,new_limit # swap the values
    iter = iter+1
    
    print(f"\n----------Iteration no.  {iter}--------------------")
    print(f"\nproduction is {produce[0][0]}")
    print(f"\nobjctive value is {first_stage_sol.value}\n")



In [55]:
import torch

In [69]:
l1 = 2
l2=  3
l3=  3
l4 = 1

# l1 = 8
# l2 = 100
# l3 = 100
# l4 = 70
# l5 = 50
# l6 = 50
# l7 = 1

model = torch.nn.Sequential(
    torch.nn.Linear(l1, l2),
    torch.nn.Tanh(),
    torch.nn.Linear(l2, l3),
    torch.nn.Tanh(),
    torch.nn.Linear(l3,l4),
)

loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
torch.nn.LeakyReLU

In [57]:
duals

array([[1.00000001],
       [1.00000002],
       [1.        ],
       [1.        ],
       [5.        ]])

In [70]:
data = []
x = 3.0
for i in range(len(all_demand)):
    data.append([x,all_demand[i][0]])

In [71]:
x_data = torch.tensor(data,dtype = torch.float32,requires_grad=True)
x_duals = torch.tensor(duals,dtype = torch.float32,requires_grad=True)
act_obj = torch.tensor(objs,dtype = torch.float32,requires_grad=True)

In [72]:
x_data

tensor([[3.0000, 1.5000],
        [3.0000, 2.0000],
        [3.0000, 3.5000],
        [3.0000, 4.0000],
        [3.0000, 5.5000]], requires_grad=True)

In [73]:
loss_hist = []
        
epochs = 10000
for epoch in range(epochs):

    model.train()
    optimizer.zero_grad()

    #Expected cost
    y_obj= model(x_data)
    loss0 = torch.mean((y_obj.squeeze() -act_obj.squeeze()) ** 2)

    d_value = torch.autograd.grad(y_obj, x_data, torch.ones_like(y_obj), create_graph=True, retain_graph=True)[0]
    dvalue_dx = d_value[:, 0].view(-1, 1)

    d2C = torch.autograd.grad(dvalue_dx, x_data, torch.ones_like(dvalue_dx), create_graph=True, retain_graph=True)[0]
    d2C_d2s = d2C[:, 0].view(-1, 1)

    loss1 = torch.mean((dvalue_dx-x_duals).squeeze())

    loss = loss0 + (10**1)*torch.abs(loss1)
    loss.backward(retain_graph=True)
    optimizer.step()

    loss_hist.append(loss.item())

    if epoch % 1 == 0:
        print(f"Epoch: {epoch} | Loss0: {loss0}, Loss1: {loss1},loss:{loss}")

    print(d2C_d2s)
    

    

Epoch: 0 | Loss0: 352.5314025878906, Loss1: -1.8014509677886963,loss:370.5458984375
tensor([[ 3.1814e-04],
        [ 1.8843e-05],
        [-2.8374e-04],
        [-3.0482e-04],
        [-2.8698e-04]], grad_fn=<ViewBackward0>)
Epoch: 1 | Loss0: 352.36566162109375, Loss1: -1.8014438152313232,loss:370.3800964355469
tensor([[ 3.4663e-04],
        [ 4.4687e-05],
        [-2.6580e-04],
        [-2.9032e-04],
        [-2.8323e-04]], grad_fn=<ViewBackward0>)
Epoch: 2 | Loss0: 352.2001037597656, Loss1: -1.8014357089996338,loss:370.2144470214844
tensor([[ 3.7507e-04],
        [ 7.0895e-05],
        [-2.4663e-04],
        [-2.7448e-04],
        [-2.7830e-04]], grad_fn=<ViewBackward0>)
Epoch: 3 | Loss0: 352.03472900390625, Loss1: -1.8014259338378906,loss:370.0489807128906
tensor([[ 4.0342e-04],
        [ 9.7437e-05],
        [-2.2624e-04],
        [-2.5730e-04],
        [-2.7214e-04]], grad_fn=<ViewBackward0>)
Epoch: 4 | Loss0: 351.8695373535156, Loss1: -1.8014150857925415,loss:369.8836975097656
te

In [27]:
y_obj[:].shape

torch.Size([5, 1])

In [20]:
d_value = torch.autograd.grad(y_obj, x_data, torch.ones_like(x_data), create_graph=True, retain_graph=True)[0]

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([5, 2]) and output[0] has a shape of torch.Size([5, 1]).

In [21]:
y_obj

tensor([[-0.5140],
        [-0.5286],
        [-0.5724],
        [-0.5870],
        [-0.6307]], grad_fn=<AddmmBackward0>)

In [52]:
x_duals

tensor([[1.],
        [1.],
        [5.],
        [5.],
        [5.]], requires_grad=True)

In [ ]:
y_obj= model(x_data)
d_value = torch.autograd.grad(y_obj, x_data, torch.ones_like(y_obj), create_graph=True, retain_graph=True)[0]
dvalue_dx = d_value[:, 0].view(-1, 1)

In [ ]:
d2C = torch.autograd.grad(dvalue_dx, x_data, torch.ones_like(dvalue_dx), create_graph=True, retain_graph=True)[0]
d2C_d2s = d2C[:, 0].view(-1, 1)

In [74]:
x_data

tensor([[3.0000, 1.5000],
        [3.0000, 2.0000],
        [3.0000, 3.5000],
        [3.0000, 4.0000],
        [3.0000, 5.5000]], requires_grad=True)

In [75]:
duals

array([[1.00000001],
       [1.00000002],
       [1.        ],
       [1.        ],
       [5.        ]])

In [77]:
y_obj

tensor([[-11.2535],
        [-12.6446],
        [-19.1998],
        [-21.1525],
        [-23.1359]], grad_fn=<AddmmBackward0>)